In [ ]:
# default_exp recommender

# Recommender

> This module exposes classes and functions related to training of the Word2Vec recommender using the Gensim library.

https://radimrehurek.com/gensim/

In [ ]:
#hide
from nbdev.export import *

# adds library to context
import sys
sys.path.append('../')

In [ ]:
#export
import logging
import random
from typing import List, NamedTuple, Tuple
from datetime import datetime
from pathlib import Path

import pandas as pd
import numpy as np

from word2vec_recommender.core import *

In [ ]:
#export
logger = logging.getLogger(__name__)

In [ ]:
#export
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize

class KnnRecommender:
    def __init__(
        self,
        word_indexes: List[str],
        embeddings: np.array,
        n_recommendations: int = 10,
        algorithm: str = 'brute'):

        self.word_indexes = word_indexes
        # https://stackoverflow.com/a/34145444 Normalize ensures euclidean will have the same output as cosine
        self.embeddings = normalize(embeddings)
        self._n_recommendations = n_recommendations

        # create and fit embeddings index
        self.nn_model: NearestNeighbors = NearestNeighbors(n_neighbors=self._n_recommendations+1, algorithm=algorithm)
        self.nn_model.fit(self.embeddings)
        
        
    def recommend_by_index(self, index: int) -> List[Recommendation]:
        if not self.nn_model:
            raise ValueError('you should call fit() before generating recommendations')
        embedding = self.embeddings[index]
        distances_array, indexes_array = self.nn_model.kneighbors([embedding])
        recommendations = []
        for ind, dist in zip(indexes_array[0][1:], distances_array[0][1:]):
            recommendations.append( Recommendation(movie_id=int(self.word_indexes[ind]), score=dist))
        return recommendations


Reading embeddings and words generated by model

In [ ]:
with open(Path("../data/best_model/embeddings.pkl"), "rb") as f:
    embeddings = np.load(f)
with open(Path("../data/best_model/words_index.pkl"), "rb") as f:
    word_indexes = np.load(f)

In [ ]:
knn_recommender = KnnRecommender(
    word_indexes=word_indexes, 
    embeddings=embeddings)


In [ ]:
movie_repository = MovieRepository(pd.read_csv('../data/ml-latest-small/movies.csv'))

Generating recommendations

> note that we need to find the index for a movie id in the embeddings array

In [ ]:
seed_id = int(word_indexes[10])
recommendations = knn_recommender.recommend_by_index(10)
print_recommendations(movie_repository, seed_id, recommendations)

Movie(movie_id=527, title="Schindler's List (1993)", genres='Drama|War')
> Recommendations:
>> Movie(movie_id=50, title='Usual Suspects, The (1995)', genres='Crime|Mystery|Thriller') score=0.9987286925315857
>> Movie(movie_id=593, title='Silence of the Lambs, The (1991)', genres='Crime|Horror|Thriller') score=1.0077788829803467
>> Movie(movie_id=318, title='Shawshank Redemption, The (1994)', genres='Crime|Drama') score=1.0118942260742188
>> Movie(movie_id=356, title='Forrest Gump (1994)', genres='Comedy|Drama|Romance|War') score=1.0190836191177368
>> Movie(movie_id=2259, title='Blame It on Rio (1984)', genres='Comedy|Romance') score=1.0370359420776367
>> Movie(movie_id=110, title='Braveheart (1995)', genres='Action|Drama|War') score=1.0436220169067383
>> Movie(movie_id=2436, title='Tea with Mussolini (1999)', genres='Comedy|Drama|War') score=1.044758915901184
>> Movie(movie_id=2028, title='Saving Private Ryan (1998)', genres='Action|Drama|War') score=1.0655676126480103
>> Movie(movie_i